In [3]:
import os

import cv2
from tqdm import tqdm

# Frame Extracter

In [ ]:
input_dir = r'E:\Dataset\Video'
output_dir = r'E:\Dataset\Frame'

In [ ]:
video_files = [f for f in os.listdir(input_dir)]

for video_name in video_files:
    video_path = os.path.join(input_dir, video_name)
    vid = cv2.VideoCapture(video_path)
    
    if not vid.isOpened():
        print(f"failed to open {video_name}")
        continue
    
    current_frame = 0
    sign_name = os.path.splitext(video_name)[0]
    data_folder = os.path.join(output_dir, sign_name)

    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    while (True):

        success, frame = vid.read()
        if not success:
            break
        
        # rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        
        cv2.imshow("output", frame)
        cv2.imwrite(os.path.join(data_folder, f"frame_{current_frame}.png"),
                                frame)
        current_frame += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vid.release()
    print(f"Completed:  {video_name} - {current_frame}")
    cv2.destroyAllWindows()

Completed:  parking_zone_raw.mp4 - 973
Completed:  priority_over_raw.mp4 - 933


# Crop to 1:1

In [14]:
input_dir = r'Dataset\Frame'
output_dir = r'Dataset\Crop'

# Vertical crop range
y1 = 420
y2 = 1500

In [13]:
img_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

for filename in tqdm(img_files, desc="Processing images"):
        
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path)

        cropped_img = img[y1:y2, :]

        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, cropped_img)

print("Done!")

Processing images:   0%|          | 0/951 [00:00<?, ?it/s]

Processing images: 100%|██████████| 951/951 [01:32<00:00, 10.32it/s]

Done!
